# Row Transformations
> Process data row by row 

In [ ]:
#| default_exp etl.national.transformations

In [ ]:
#| export
import os
import pathlib
import pandas as pd
import unidecode

from niloIQ2.etl import tables

In [ ]:
#| export

keep_columns = [
    'producto', 'origen', 'destino', 'date', 'precio_minimo', 'precio_maximo',
    'precio_frecuente'
]

new_column_names = [
    'product', 'source', 'dest', 'date', 'min_price', 'max_price', 'price',
]

 
old_version = '02'
new_version = '03'
for table in tables.national_list:

    df = pd.read_csv(table.version(old_version),low_memory=False)
    df = df[keep_columns]
    
    df.columns = new_column_names
    

    df['state'] = df['dest'].apply(lambda x: x.split(':')[0].strip())
    df['state'] = df['state'].apply(
        lambda x: unidecode.unidecode(x) if x != 'DF' else 'Ciudad de Mexico'
    )

    df['center'] = df['dest'].apply(lambda x: x.split(':')[-1])

    df['import'] = df['source'].apply(lambda x: x == 'Importación')

    #df.drop(columns=['destino'],inplace=True)

    df['min_price'] = pd.to_numeric(df['min_price'], errors='coerce')
    df['max_price'] = pd.to_numeric(df['max_price'], errors='coerce')
    df['price'] = pd.to_numeric(df['price'], errors='coerce')


    # remove mixed types in min and max prices substiting them with price
    df['min_price'].mask(df['min_price'].isnull(), df['price'], inplace=True) 
    df['max_price'].mask(df['max_price'].isnull(), df['price'], inplace=True) 


    df.to_csv(table.version(new_version), index=False)

### Tests

In [ ]:

## There are no supply centers in Tlaxcala

all_states = set(
    [
        'Aguascalientes', 'Baja California', 'Baja California Sur', 'Campeche',
        'Chiapas', 'Chihuahua', 'Coahuila', 'Colima', 'Ciudad de Mexico',
        'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Mexico',
        'Michoacan', 'Morelos', 'Nayarit', 'Nuevo Leon', 'Oaxaca', 'Puebla',
        'Queretaro', 'Quintana Roo', 'San Luis Potosi', 'Sinaloa', 'Sonora',
        'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatan', 'Zacatecas'
    ]
)

for table in tables.national_list:

    old_df = pd.read_csv(table.version(old_version), low_memory=False)
    new_df = pd.read_csv(
        table.version(new_version)
    )  # low_memory=True, all columns are uniform

    states = set(
        new_df['state'].value_counts().index.values
    )  # Mexican states in data

    #all True values in import column are 'Importacion'  in origen column
    assert states.issubset(all_states)
    assert len(states) == 31
    assert 'Tlaxcala' not in states

    #all True values in import column are 'Importacion'  in origen column
    assert all(old_df[new_df['import']]['origen'] == 'Importación')

    if 'historica' in table.table_name:
        # old historica have mixed values
        assert len(old_df[old_df['precio_maximo'] == '--']) == 5  
        assert len(old_df[old_df['precio_minimo'] == '--']) == 5  
    else: 
        # old actual is ok
        assert len(old_df[old_df['precio_maximo'] == '--']) == 0  
        assert len(old_df[old_df['precio_minimo'] == '--']) == 0 


    # new version is float type (not mixed)
    assert new_df['min_price'].dtype == float
    assert new_df['max_price'].dtype == float

    # '--' values in historica were replaced by prices

    from pandas.testing import assert_series_equal

    masked_df = new_df[old_df['precio_minimo'] == '--']

    assert_series_equal(masked_df['min_price'], masked_df['price'], check_names=False)
    assert_series_equal(masked_df['max_price'], masked_df['price'], check_names=False)

